<a href="https://colab.research.google.com/github/chaos44/generative_deep_learning/blob/main/chap3/3_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone -b tensorflow_2 https://github.com/davidADSP/GDL_code.git

fatal: destination path 'GDL_code' already exists and is not an empty directory.


In [31]:
!pip install -r requirements.txt

     |████████████████████████████████| 112kB 8.0MB/s 
     |████████████████████████████████| 40kB 6.8MB/s 
     |████████████████████████████████| 163kB 11.7MB/s 
     |████████████████████████████████| 92kB 7.4MB/s 
     |████████████████████████████████| 3.3MB 13.5MB/s 
     |████████████████████████████████| 122kB 51.5MB/s 
     |████████████████████████████████| 778kB 53.2MB/s 
     |████████████████████████████████| 1.0MB 22.8MB/s 
     |████████████████████████████████| 133kB 50.9MB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
     |████████████████████████████████| 92kB 14.2MB/s 
     |████████████████████████████████| 378kB 51.7MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
     |████████████████████████████████| 92kB 14.1MB/s 
     |████████████████████████████████| 13.1MB 251kB/s 
     |

In [2]:
cd GDL_code/

/content/GDL_code


## **Autoencoder**

In [32]:
import os

from utils.loaders import load_mnist
from models.AE import Autoencoder

[autoreload of six failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    del self.failed[py_filename]
AttributeError: 'NoneType' object has no attribute 'filter'
]
[autoreload of IPython.core.ultratb failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    del self.failed[py_filename]
AttributeError: module 'IPython' has no attribute 'utils'
]
[autoreload of pygments failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    del self.failed[py_filename]
ImportError: cannot import name 'StringIO'
]


# **Set parameters**

In [34]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

# **Load the data**

In [35]:
(x_train, y_train), (x_test, y_test) = load_mnist()

# **Define the structure of the neural network**

In [36]:
AE = Autoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if MODE == 'build':
    AE.save(RUN_FOLDER)
else:
    AE.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [37]:
AE.encoder.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 28, 28, 1)]       0         
_________________________________________________________________
encoder_conv_0 (Conv2D)      (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
encoder_conv_1 (Conv2D)      (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
encoder_conv_2 (Conv2D)      (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_23 (LeakyReLU)   (None, 7, 7, 64)         

In [38]:
AE.decoder.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_3 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 14, 14, 64)       

# **Train the autoencoder**

In [39]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0

In [40]:
AE.compile(LEARNING_RATE)

In [41]:
AE.train(     
    x_train[:1000]
    , batch_size = BATCH_SIZE
    , epochs = 200
    , run_folder = RUN_FOLDER
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/200
29/32 [==========================>...] - ETA: 0s - loss: 0.1709
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 9ms/step - loss: 0.1654
Epoch 2/200
25/32 [======================>.......] - ETA: 0s - loss: 0.0756
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 7ms/step - loss: 0.0742
Epoch 3/200
29/32 [==========================>...] - ETA: 0s - loss: 0.0666
Epoch 00003: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.0664
Epoch 4/200
29/32 [==========================>...] - ETA: 0s - loss: 0.0615
Epoch 00004: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [==============================] - 0s 6ms/step - loss: 0.0613
Epoch 5/200
31/32 [============================>.] - ETA: 0s - loss: 0.0588
Epoch 00005: saving model to run/vae/0002_digits/weights/weights.h5
32/32 [=

# **VAE Analysis**
## **imports**

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist, load_model

ModuleNotFoundError: ignored

## **imports**

## **architecture**

In [3]:
vae = load_model(VariationalAutoencoder, RUN_FOLDER)

NameError: ignored